In [6]:
import sys
import pandas as pd
from pathlib import Path
if '..' not in sys.path:
    sys.path.append('..')
    
import dataset

In [40]:
submits = []
for path in sorted(Path('../../_data/voting/').glob('*.csv')):
    print(path.name)
    submits.append(pd.read_csv(path, index_col=0))

07-20170720-232049-b7-albu-0.93374-03-albu-mean3_0.933493-bayes.csv
20170720-005737-b6-albu-0.93359-bayes-test.csv
20170720-005737-b6-albu-0.93359.csv
20170720-005737-b6-albu-0.93359_weather_hack0606.csv
20170720-232049-b7-albu-0.93374.csv


In [42]:
scores = [0.93398, 0.93396, 0.93392, 0.93401, 0.93392]

In [5]:
submits[0][:3]

,tags
image_name,
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary


In [14]:
def to_flat(df):
    flat = pd.DataFrame(0, index=df.index, columns=dataset.CLASSES)
    for i, tags in enumerate(df['tags']):
        row = flat.iloc[i]
        for cls in tags.split(' '):
            row[cls] = 1
    return flat

In [41]:
flat_submits = [to_flat(x) for x in submits]

In [19]:
mean = sum(flat_submits) / 5
mean[:3]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
test_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
test_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [44]:
submits[0][:3]

,tags
image_name,
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary


In [39]:
((mean != 0) & (mean != 1)).any(1).sum()

3167

In [33]:
vote_2 = mean >= (2/5 - 1e-5)
vote_3 = mean >= (3/5 - 1e-5)

In [38]:
(vote_2 != vote_3).any(1).sum()

500

In [57]:
skip = {2}
w_mean = (sum(df * w for i, (df, w) in enumerate(zip(flat_submits, scores)) if i not in skip) /
          sum(s for i, s in enumerate(scores) if i not in skip))
vote_fair = w_mean > 0.5

In [65]:
(vote_2 != vote_fair).any(1).sum()

1239

In [66]:
(vote_3 != vote_fair).any(1).sum()

750

In [67]:
vote_2.sum()

agriculture          23980
artisinal_mine         622
bare_ground           2059
blooming               530
blow_down              118
clear                46006
cloudy                4959
conventional_mine      143
cultivation          10341
habitation            7347
haze                  5734
partly_cloudy        12821
primary              58408
road                 14916
selective_logging      666
slash_burn             313
water                13797
dtype: int64

In [69]:
vote_3.sum()

agriculture          23941
artisinal_mine         614
bare_ground           2041
blooming               523
blow_down              114
clear                45967
cloudy                4941
conventional_mine      141
cultivation          10295
habitation            7309
haze                  5687
partly_cloudy        12761
primary              58372
road                 14845
selective_logging      660
slash_burn             304
water                13734
dtype: int64

In [70]:
vote_fair.sum()

agriculture          23856
artisinal_mine         614
bare_ground           2010
blooming               491
blow_down              110
clear                45902
cloudy                4901
conventional_mine      141
cultivation          10160
habitation            7266
haze                  5608
partly_cloudy        12711
primary              58340
road                 14790
selective_logging      644
slash_burn             293
water                13658
dtype: int64

In [76]:
flat_submits[-2].sum()

agriculture          23927
artisinal_mine         615
bare_ground           2055
blooming               528
blow_down              116
clear                45969
cloudy                4950
conventional_mine      142
cultivation          10321
habitation            7338
haze                  5628
partly_cloudy        12737
primary              58240
road                 14883
selective_logging      665
slash_burn             313
water                13721
dtype: int64

In [77]:
flat_submits[0].sum()

agriculture          23779
artisinal_mine         619
bare_ground           1964
blooming               465
blow_down              109
clear                45813
cloudy                4910
conventional_mine      141
cultivation          10065
habitation            7157
haze                  5585
partly_cloudy        12653
primary              58346
road                 14657
selective_logging      617
slash_burn             244
water                13576
dtype: int64

In [72]:
w_mean[((w_mean > 0.5) & (w_mean < 0.7)).any(1)][:10]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_1670,1.000000,0.000000,0.500009,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,1.000000
test_3047,1.000000,0.500009,1.000000,0.0,0.0,1.000000,0.0,0.0,1.0,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0,1.000000
test_3179,0.500009,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,1.0,1.000000,0.0,0.0,1.000000
test_3430,1.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.500009,0.0,0.0,1.0,1.000000,0.0,0.0,0.000000
test_3954,1.000000,0.499991,1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.500001,1.0,0.0,1.000000
test_5049,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.500009,0.0,0.0,1.000000
test_5758,0.000000,0.000000,0.000000,0.0,0.0,0.500009,0.0,0.0,0.0,1.000000,0.0,1.0,1.0,1.000000,0.0,0.0,1.000000
test_6041,1.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,1.0,0.500009,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000
test_6543,1.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.000000,0.0,0.0,0.500009


In [71]:
w_mean[(vote_3 != vote_fair).any(1)][:10]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_178,0.0,0.0,0.0,0.0,0.0,0.0,0.499999,0.0,0.0,0.000000,1.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000
test_193,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.499999,0.0,1.000000
test_353,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.499985
test_430,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,1.0,0.499999,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000
test_457,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,1.0,0.499999,0.000000,0.0,0.000000
test_508,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,1.0,0.499999,0.000000,0.0,1.000000
test_591,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.499999,0.000000,0.0,1.0,1.000000,0.000000,0.0,0.000000
test_606,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.499999,0.000000,0.0,1.0,1.000000,0.000000,0.0,0.000000
test_766,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.499999,0.0,1.0,0.000000,0.000000,0.0,0.000000


In [73]:
submit = vote_fair
out_df = pd.DataFrame([
    {'tags': ' '.join(c for c in submit.columns if row[c])
     } for image_name, row in submit.iterrows()],
    index=submit.index)

In [74]:
out_df.to_csv('../../vote_fair.csv', index_label='image_name')
! head ../../vote_fair.csv

image_name,tags
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary
test_3,agriculture clear cultivation primary
test_4,cloudy partly_cloudy primary
test_5,clear primary
test_6,agriculture clear cultivation habitation partly_cloudy primary
test_7,agriculture clear habitation primary road
test_8,clear primary
